# Prueba

In [1]:
import psycopg2 # Se conecta a postgre
import pandas.io.sql as pdsql # transforma las salidas de postgre en formato pandas dataframe
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
clave = 'Insert_Clave'

## 1. 
Genera una función llamada leer_tabla(tabla, engine) y utilízala para leer tablas
 completas desde la base de datos en dataframes independientes. Utilizando esta
 función, importa las siguientes tablas:
 - order
 - orderdetails
 - customers
 - products
 - employees

In [3]:
#Creamos el objeto engine:
engine = psycopg2.connect(user ='postgres',
                                 password = clave,
                                 host = '127.0.0.1',
                                 port = 5432,
                                 database = 'classicmodels')

In [4]:
#generamos la funcion leer_tabla
def leer_tabla(tabla, engine): 

    df = pdsql.read_sql_query(f"SELECT * FROM {tabla}", engine)

    return df

In [5]:
leer_tabla('orders',engine)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,None,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,None,141
...,...,...,...,...,...,...,...
321,10421,2005-05-29,2005-06-06,None,In Process,Custom shipping instructions were sent to ware...,124
322,10422,2005-05-30,2005-06-11,None,In Process,None,157
323,10423,2005-05-30,2005-06-05,None,In Process,None,314
324,10424,2005-05-31,2005-06-08,None,In Process,None,141


## Usamos leer_tabla para leer tablas
## completas desde la base de datos en dataframes independientes. Utilizando esta
## función, importa las siguientes tablas:

In [6]:
df_orders = leer_tabla('orders',engine)
df_orderdetails = leer_tabla('orderdetails',engine)
df_customers = leer_tabla('customers',engine)
df_products = leer_tabla('products',engine)
df_employees = leer_tabla('employees',engine)


In [7]:
df_orders.head(3)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181


In [8]:
df_orders[df_orders['orderNumber'] == 10100] #Se detectan ordenes duplicadas

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363


In [9]:
# Se eliminan los duplicados
df_orders_unq = df_orders.drop_duplicates()
df_orders_unq.shape

(326, 7)

In [10]:
df_orders_unq.head(3) #revision del df generado con leer_tabla

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181


In [11]:
df_orderdetails.head(3) #revision del df generado con leer_tabla

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4


In [12]:
df_customers.head(3) #revision del df generado con leer_tabla

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0


In [13]:
df_customers.customerNumber.value_counts()  #Se detectan ordenes duplicadas
df_customers[df_customers['customerNumber'] == 103]

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0


In [14]:
# Se eliminan los duplicados

df_customers.rename(columns={'salesRepEmployeeNumber': 'employeeNumber'}, inplace=True)
df_customers_unq = df_customers.drop_duplicates()
df_customers_unq.shape

(122, 13)

In [15]:
df_products.head(3) #Revision del df desde leer_tabla

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94


In [16]:
df_products_unq = df_products.drop_duplicates() # Se eliminan los duplicados
df_products_unq.shape

(110, 9)

In [17]:

df_employees_unq = df_employees.drop_duplicates() # Se eliminan los duplicados
df_employees_unq.shape

(23, 8)

In [18]:
df_employees.head(3)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing


## 2. 
Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el
 parámetro validate para asegurar la integridad referencial.


In [19]:
print(df_customers_unq.shape)
df_base = df_customers_unq.merge(df_orders_unq, on='customerNumber', how='left', validate='1:m') #Se usa le f merge para generar el df_base uniendo orders_unq con costumerNumber haciendo leftjoin y validando 1:m
print(df_base.shape)

(122, 13)
(350, 19)


In [20]:
df_base = df_base.merge(df_orderdetails, on='orderNumber', how='left', validate='m:m') #Se usa la f merge sobre df_base uniendo el df anterior con order_Details y se revisa el shape
print(df_base.shape)

(3020, 23)


In [21]:
df_base = df_base.merge(df_products_unq, on='productCode', how='left', validate='m:1') #Se usa la f merge sobre df_base uniendo el df anterior con df_products_unq y se revisa la integridad con shape
print(df_base.shape)

(3020, 31)


In [22]:
df_base = df_base.merge(df_employees_unq, on='employeeNumber', how='left', validate='m:1') #Se usa la f merge sobre df_base uniendo el df anterior con df_employees_unq y se revisa la integridad con shape
print(df_base.shape) 

(3020, 38)


In [23]:
df_base.columns #Se revisan las columnas agregadas

Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'employeeNumber', 'creditLimit', 'orderNumber', 'orderDate',
       'requiredDate', 'shippedDate', 'status', 'comments', 'productCode',
       'quantityOrdered', 'priceEach', 'orderLineNumber', 'productName',
       'productLine', 'productScale', 'productVendor', 'productDescription',
       'quantityInStock', 'buyPrice', 'MSRP', 'lastName', 'firstName',
       'extension', 'email', 'officeCode', 'reportsTo', 'jobTitle'],
      dtype='object')

# 3. 
Agrega las siguientes columnas, considerando su nombre y la fórmula asociada
 - venta: quantityOrdered*priceEach
 - costo: quantityOrdered*buyPrice
 - ganancia: considerando las columnas anteriores


In [24]:
df_base['venta'] = df_base['quantityOrdered'] * df_base['priceEach']
df_base['costo'] = df_base['quantityOrdered'] * df_base['buyPrice']
df_base['ganancia'] = df_base['venta'] - df_base['costo'] #Se agregan las columnas venta costo y ganancia

In [25]:
df_base.columns #se revisan las columnas nuevas

Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'employeeNumber', 'creditLimit', 'orderNumber', 'orderDate',
       'requiredDate', 'shippedDate', 'status', 'comments', 'productCode',
       'quantityOrdered', 'priceEach', 'orderLineNumber', 'productName',
       'productLine', 'productScale', 'productVendor', 'productDescription',
       'quantityInStock', 'buyPrice', 'MSRP', 'lastName', 'firstName',
       'extension', 'email', 'officeCode', 'reportsTo', 'jobTitle', 'venta',
       'costo', 'ganancia'],
      dtype='object')

## 4. ¿Cuál fue el total de ventas por línea de productos? Incluye una fila de totales.

In [26]:
# Se agrupa por línea de productos y suma las ventas
total_ventas_por_linea = df_base.groupby('productLine')['venta'].sum().reset_index(name='total_ventas')

# Se agrega una columna de totales al DataFrame original
df_base = pd.merge(df_base, total_ventas_por_linea, on='productLine', how='left')
print(f"total de ventas por linea de productos: {total_ventas_por_linea}")

total de ventas por linea de productos:         productLine  total_ventas
0      Classic Cars    3853922.49
1       Motorcycles    1121426.12
2            Planes     954637.54
3             Ships     663998.34
4            Trains     188532.92
5  Trucks and Buses    1024113.57
6      Vintage Cars    1797559.63


In [27]:
df_base.columns #se revisan las columnas

Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'employeeNumber', 'creditLimit', 'orderNumber', 'orderDate',
       'requiredDate', 'shippedDate', 'status', 'comments', 'productCode',
       'quantityOrdered', 'priceEach', 'orderLineNumber', 'productName',
       'productLine', 'productScale', 'productVendor', 'productDescription',
       'quantityInStock', 'buyPrice', 'MSRP', 'lastName', 'firstName',
       'extension', 'email', 'officeCode', 'reportsTo', 'jobTitle', 'venta',
       'costo', 'ganancia', 'total_ventas'],
      dtype='object')

## 5. ¿Cuántos clientes distintos hicieron compras?

In [28]:
#Se filtran aquellos Números de orden no nulos 
#y luego de cuentan aquellos clientes unicos asociados a los números de orden
clientes_diferentes = df_base[~ df_base['orderNumber'].isnull()]['customerName'].nunique()
print(f"Número de clientes diferentes que hicieron compras: {clientes_diferentes}")

Número de clientes diferentes que hicieron compras: 98


## 6. ¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?

In [29]:
clientes_sin_compras = df_base[df_base['orderNumber'].isnull()]

if clientes_sin_compras.empty:
    print("Todos los clientes han realizado compras.")
else:
    print("Existen clientes que aún no han hecho ninguna compra.")

# Para determinar cuántos son:
cantidad_clientes_sin_compras = len(clientes_sin_compras)
print(f"Número de clientes sin compras: {cantidad_clientes_sin_compras}")

Existen clientes que aún no han hecho ninguna compra.
Número de clientes sin compras: 24


 ¿Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante
 el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
top_10_clientes_2005, en la que se especifique el nombre del cliente y su
 correspondiente venta, costo y ganancia.

In [30]:
#Llamamos las funciones presentes en funciones.py
from funciones import filtro_fecha_orden

In [31]:
#Filtramos el año 2005

df2005=filtro_fecha_orden(df_base, '2005-01-01', '2005-12-31')


In [32]:
print(df2005['orderDate'].min()) #prueba para revisiar la aplicacion del filtro
print(df2005['orderDate'].max())

2005-01-05 00:00:00
2005-05-31 00:00:00


In [33]:
#10 clientes que reportan mayores ventas brutas en dinero durante el año 2005

#sumamos las ventas por cliente
from funciones import suma_por_var

ven_cli=suma_por_var(df_base, 'customerName', 'venta')
ven_cli

,customerName,venta
0,ANG Resellers,0.00
1,"AV Stores, Co.",148410.09
2,Alpha Cognac,60483.36
3,American Souvenirs Inc,0.00
4,Amica Models & Co.,82223.23
...,...,...
117,Vitachrome Inc.,72497.64
118,"Volvo Model Replicas, Co",66694.82
119,Warburg Exchange,0.00
120,West Coast Collectables Co.,43748.72


In [34]:
ven_cli.sort_values(by='venta', ascending=False, inplace=True) # Ordena el df descendente
top_10_customers = ven_cli.head(10) # Selecciona los 10 superiores
top_10_customers

,customerName,venta
42,Euro+ Shopping Channel,820689.54
76,Mini Gifts Distributors Ltd.,591827.34
11,"Australian Collectors, Co.",180585.07
80,Muscle Machine Inc,177913.95
63,La Rochelle Gifts,158573.12
40,"Dragon Souveniers, Ltd.",156251.03
39,"Down Under Souveniers, Inc",154622.08
64,Land of Toys Inc.,149085.15
1,"AV Stores, Co.",148410.09
110,The Sharp Gifts Warehouse,143536.27


¿Cuál fue el top 10 de artículos más vendidos durante el año 2005 (considerando
cantidad neta)? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
top_10_productos_2005, en la que se especifique el nombre del producto y su
correspondiente venta, costo y ganancia.


In [35]:
#Llamamos las funciones presentes en funciones.py
from funciones import filtro_fecha_orden

In [36]:
#Filtramos el año 2005

df2005=filtro_fecha_orden(df_base, '2005-01-01', '2005-12-31')


In [37]:
#Revisamos el df filtrado
print(df2005['orderDate'].min()) 
print(df2005['orderDate'].max())

2005-01-05 00:00:00
2005-05-31 00:00:00


In [39]:
#10 de artículos más vendidos durante el año 2005

#sumamos las cantidad neta por articulo
from funciones import suma_por_var

sum_art=suma_por_var(df_base, 'productName', 'quantityOrdered')
sum_art

,productName,quantityOrdered
0,18th Century Vintage Horse Carriage,907
1,18th century schooner,1011
2,1900s Vintage Bi-Plane,940
3,1900s Vintage Tri-Plane,1009
4,1903 Ford Model A,883
...,...,...
104,The Mayflower,898
105,The Queen Mary,896
106,The Schooner Bluenose,934
107,The Titanic,952


Una función que permita generar reportes dependiendo de parámetros de entrada
como dataframe, filas, columnas, valores y medida (funcion_agrupadora). Utilizar
fill_value = 0. Esta función debe retornar un DataFrame pivotado.

In [40]:
sum_art.sort_values(by='quantityOrdered', ascending=False, inplace=True) # Ordena el df descendente
top_10_articles = sum_art.head(10) # Selecciona los 10 superiores
top_10_articles

,productName,quantityOrdered
76,1992 Ferrari 360 Spider red,1808
24,1937 Lincoln Berline,1111
95,American Airlines: MD-11S,1085
31,1941 Chevrolet Special Deluxe Cabriolet,1076
15,1930 Buick Marquette Phaeton,1074
30,1940s Ford truck,1061
61,1969 Harley Davidson Ultimate Chopper,1057
40,1957 Chevy Pickup,1056
51,1964 Mercedes Tour Bus,1053
39,1956 Porsche 356A Coupe,1052


 Una función que permita escribir en la base de datos a través del guardado de un
 DataFrame dependiendo de parámetros de entrada como DataFrame, nombre de la
 tabla, engine y comportamiento en caso de que exista la tabla (if_exists)

In [44]:
from funciones import dataframeToPostgresql #Llama a la funcion

# Datos de conexión
conexion = {
        "usuario": "postgres",
        "contraseña": "CLAVE",
        "host": "127.0.0.1",
        "puerto": "5432",
        "base_de_datos": "classicmodels"
    }

conn_string = f"postgresql://{conexion['usuario']}:{conexion['contraseña']}@{conexion['host']}:{conexion['puerto']}/{conexion['base_de_datos']}"


# Nombre de la tabla y esquema
table_name = "top_10_articles"
esquema = "public"
df = top_10_articles
# Guardar el DataFrame en PostgreSQL
dataframeToPostgresql(df, table_name, conn_string, esquema)

Tabla 'top_10_articles' creada y datos insertados.
